# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Construct-Configuration-Matrices" data-toc-modified-id="Construct-Configuration-Matrices-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Construct Configuration Matrices</a></div><div class="lev1 toc-item"><a href="#Optimize-Dynamic-Subgraphs" data-toc-modified-id="Optimize-Dynamic-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optimize Dynamic Subgraphs</a></div><div class="lev2 toc-item"><a href="#NMF-Cross-Validation-Optimizaion" data-toc-modified-id="NMF-Cross-Validation-Optimizaion-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>NMF Cross-Validation Optimizaion</a></div><div class="lev2 toc-item"><a href="#Quality-Measures-in-Parameter-Space" data-toc-modified-id="Quality-Measures-in-Parameter-Space-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Quality Measures in Parameter Space</a></div><div class="lev1 toc-item"><a href="#Detect-Dynamic-Subgraphs" data-toc-modified-id="Detect-Dynamic-Subgraphs-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Detect Dynamic Subgraphs</a></div><div class="lev2 toc-item"><a href="#Run-Non-Negative-Matrix-Factorization-Algorithm" data-toc-modified-id="Run-Non-Negative-Matrix-Factorization-Algorithm-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Run Non-Negative Matrix Factorization Algorithm</a></div><div class="lev2 toc-item"><a href="#Consensus-Clustering-of-Dynamic-Subgraphs" data-toc-modified-id="Consensus-Clustering-of-Dynamic-Subgraphs-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Consensus Clustering of Dynamic Subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-Subgraphs" data-toc-modified-id="Plot-Subgraphs-321"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Plot Subgraphs</a></div><div class="lev1 toc-item"><a href="#Subgraphs-of-Brain-Systems" data-toc-modified-id="Subgraphs-of-Brain-Systems-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Subgraphs of Brain Systems</a></div><div class="lev2 toc-item"><a href="#Load-Subgraphs-and-Expression" data-toc-modified-id="Load-Subgraphs-and-Expression-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load Subgraphs and Expression</a></div><div class="lev2 toc-item"><a href="#Load-Atlas" data-toc-modified-id="Load-Atlas-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Load Atlas</a></div><div class="lev2 toc-item"><a href="#Convert-ROI-Subgraphs-to-Brain-System-Subgraphs" data-toc-modified-id="Convert-ROI-Subgraphs-to-Brain-System-Subgraphs-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Convert ROI Subgraphs to Brain System Subgraphs</a></div><div class="lev2 toc-item"><a href="#Filter-Subgraphs-with-Sparse-Expression" data-toc-modified-id="Filter-Subgraphs-with-Sparse-Expression-44"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Filter Subgraphs with Sparse Expression</a></div><div class="lev2 toc-item"><a href="#Plot-all-Brain-System-Subgraphs" data-toc-modified-id="Plot-all-Brain-System-Subgraphs-45"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Plot all Brain System Subgraphs</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/data/jag/akhambhati/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr
nmf = Echobase.Network.Partitioning.Subgraph.nmf

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_Remotes = '/data/jag/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_InpData = path_PeriphData + '/e01-FuncNetw'
path_ExpData = path_PeriphData + '/e02b-FuncSubg'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Generate List of Data

In [ ]:
inp_fname = glob.glob('{}/*.npz'.format(path_InpData))

expr_dict = {}
for fname in inp_fname:
    subj_id = fname.split('/')[-1].split('.')[0]
    expr_id = fname.split('/')[-1].split('.')[1]
    
    try:
        expr_dict[expr_id]['adj_files'].append(fname)
    except:
        expr_dict[expr_id] = {}
        expr_dict[expr_id]['adj_files'] = []
        expr_dict[expr_id]['adj_files'].append(fname)

## Construct Configuration Matrices
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
key_ord = ['adj_rs_pos', 'adj_rs_neg',
           'adj_lo_pos', 'adj_lo_neg',
           'adj_hi_pos', 'adj_hi_neg']

cfg_list = []
cfg_key = []
for expr_id in expr_dict.keys():    
    for fname in expr_dict[expr_id]['adj_files']:
        df = np.load(fname)
        
        for key in key_ord:
            cfg_matr = convert_adj_matr_to_cfg_matr(df[key])
            for cfg_vec in cfg_matr:
                cfg_list.append(cfg_vec)
                cfg_key.append('{}_{}'.format(key, expr_id))
cfg_matr=np.array(cfg_list)
cfg_key=np.array(cfg_key)

np.savez('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData),
         cfg_matr=cfg_matr,
         cfg_key=cfg_key)

# Optimize Dynamic Subgraphs

## NMF Cross-Validation Optimizaion

In [ ]:
# Load configuration matrix
cfg_data = np.load('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))
cfg_matr = cfg_data['cfg_matr']

# Set search params
search_alpha = list(np.random.uniform(low=0.01, high=1.0, size=1000))
search_beta = list(np.random.uniform(low=0.01, high=1.0, size=1000))
search_rank = list(np.random.randint(low=2, high=61, size=1000))
search_fold = 10

# Cross-Validation Optimization
str_path = '{}/NMF_Optimization.Error.txt'.format(path_ExpData)
if os.path.exists(str_path):
    os.remove(str_path)
    
param_list = Echobase.Network.Partitioning.Subgraph.optimize_nmf.cross_validation(
    cfg_matr, search_alpha, search_beta, search_rank, search_fold, n_proc=14,
    str_path=str_path)

np.savez('{}/NMF_Optimization.Error.npz'.format(path_ExpData),
         alpha=param_list['alpha'],
         beta=param_list['beta'],
         rank=param_list['rank'],
         error=param_list['error'],
         pct_sparse_subgraph=param_list['pct_sparse_subgraph'],
         pct_sparse_coef=param_list['pct_sparse_coef'])

## Quality Measures in Parameter Space

In [ ]:
opt_dict = np.load('{}/NMF_Optimization.Error.npz'.format(path_ExpData), mmap_mode='r')
#opt_params = Echobase.Network.Partitioning.Subgraph.optimize_nmf.min_crossval_param(dict(opt_dict))

error_ix = np.flatnonzero(opt_dict['error'] < np.percentile(opt_dict['error'], 25))
sparse_subg_ix = np.flatnonzero(opt_dict['pct_sparse_subgraph'] > np.percentile(opt_dict['pct_sparse_subgraph'], 75))
sparse_coef_ix = np.flatnonzero(opt_dict['pct_sparse_coef'] > np.percentile(opt_dict['pct_sparse_coef'], 75))
joint_ix = np.intersect1d(np.intersect1d(error_ix, sparse_subg_ix), sparse_coef_ix)

opt_params = {}
opt_params['rank'] = int(opt_dict['rank'][joint_ix].mean().round())
opt_params['alpha'] = opt_dict['alpha'][joint_ix].mean()
opt_params['beta'] = opt_dict['beta'][joint_ix].mean()
print('Optimal Rank: {}'.format(opt_params['rank']))
print('Optimal Alpha: {}'.format(opt_params['alpha']))
print('Optimal Beta: {}'.format(opt_params['beta']))

# Generate quality measure plots
for qmeas in ['error', 'pct_sparse_subgraph', 'pct_sparse_coef']:
    for param in ['rank', 'alpha', 'beta']:

        ax_jp = sns.jointplot(opt_dict[param], opt_dict[qmeas], kind='kde', space=0, n_levels=5, shade_lowest=False)
        ax = ax_jp.ax_joint
        ax.plot([opt_params[param], opt_params[param]], 
                [ax.get_ylim()[0]*1.05, ax.get_ylim()[1]*0.95],
                lw=1.0)

        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.set_xlabel(param)
        ax.set_ylabel(qmeas)
        
        #plt.savefig('./e02c-Figures/NMF_Optimization.{}.{}.svg'.format(param, qmeas))
        plt.show()
        plt.close()
opt_dict.close()

# Detect Dynamic Subgraphs

## Run Non-Negative Matrix Factorization Algorithm
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/*.subgraph_seed-*.npz".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
param = {'rank': 51,
         'alpha': 0.327,
         'beta': 0.577}
n_seed = 100

from multiprocessing import Pool
parallel_run = True

# Generate a processing joblist
cfg_matr_path = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))[0]
proc_list = []
for seed in xrange(n_seed):
    proc_list.append({'path': cfg_matr_path,
                      'param': param,
                      'seed': seed+1})
    
# Setup helper function to map pipeline run
def _nmf_helper(proc_item):
    
    # Load the file
    #if os.path.exists(inp_path):
    #    return 0
    print(" -- Processing Seed: {}".format(proc_item['seed']))
    data = np.load(proc_item['path'], mmap_mode='r')
    
    # Initialize the factors for NMF
    fac_subnet = np.random.uniform(low=0, high=1.0,
                                   size=(proc_item['param']['rank'],
                                         data['cfg_matr'].shape[1]))
    fac_coef = np.random.uniform(low=0, high=1.0,
                                 size=(proc_item['param']['rank'],
                                       data['cfg_matr'].shape[0]))

    # Run NMF Algorithm
    fac_subnet, fac_coef, err = nmf.snmf_bcd(
        data['cfg_matr'],
        alpha=proc_item['param']['alpha'],
        beta=proc_item['param']['beta'],
        fac_subnet_init=fac_subnet,
        fac_coef_init=fac_coef,
        max_iter=100, verbose=False)
    
    # Cache the NMF result
    np.savez("{}/NMF_Optimization.subgraph_seed-{}.npz".format(path_ExpData,
                                                               proc_item['seed']),
             fac_subnet=fac_subnet, fac_coef=fac_coef, err=err,
             param=proc_item['param'], path=proc_item['path'])

if parallel_run:
    mp = Pool(10)
    mp.map(_nmf_helper, proc_list)
else:
    map(_nmf_helper, proc_list)

## Consensus Clustering of Dynamic Subgraphs

In [ ]:
seed_paths = glob.glob("{}/NMF_Optimization.subgraph_seed-*.npz".format(path_ExpData))

# Aggregate the estimated subgraphs of each seed
fac_subnet_seeds = []
for ii, path in enumerate(seed_paths):
    data = np.load(path, mmap_mode='r')
    fac_subnet = data['fac_subnet'][:, :]
    data.close()

    n_fac = fac_subnet.shape[0]
    n_conn = fac_subnet.shape[1]

    for iy in xrange(fac_subnet.shape[0]):
        fac_subnet_seeds.append(fac_subnet[iy, :])
fac_subnet_seeds = np.array(fac_subnet_seeds)

n_obs = fac_subnet_seeds.shape[0]
n_conn = fac_subnet_seeds.shape[1]

# Consensus Subgraphs
fac_cons_subnet, fac_cons_seeds, err = nmf.snmf_bcd(
    fac_subnet_seeds,
    alpha=0.0,
    beta=0.0,
    fac_subnet_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_conn)),
    fac_coef_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_obs)),
    max_iter=100, verbose=False)

# Consensus Coefficients
cfg_matr_path = glob.glob("{}/NMF_Optimization.CfgMatr.npz".format(path_ExpData))[0]
data_cfg = np.load(cfg_matr_path, mmap_mode='r')
n_win = data_cfg['cfg_matr'].shape[0]
fac_cons_subnet_2, fac_cons_coef_2, err = nmf.snmf_bcd(
    data_cfg['cfg_matr'],
    alpha=0.0,
    beta=0.0,
    fac_subnet_init=fac_cons_subnet,
    fac_coef_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_win)),
    max_iter=100, verbose=False)

# Cache the Consensus NMF result
np.savez("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
         fac_subnet=fac_cons_subnet_2, fac_coef=fac_cons_coef_2, err=err)

### Plot Subgraphs

In [ ]:
%matplotlib inline

# Load the consensus data
data = np.load("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
               mmap_mode='r')
fac_subnet = data['fac_subnet']
fac_coef = data['fac_coef']

# Normalize
fac_subnet = fac_subnet / fac_subnet.max()
fac_coef = fac_coef / fac_coef.max()

n_fac = fac_subnet.shape[0]
n_conn = fac_subnet.shape[1]
n_win = fac_coef.shape[1]

# Plot subgraph matrix
plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(fac_subnet.T, aspect=n_fac/n_conn, cmap='rainbow', vmin=0, vmax=1)
plt.colorbar(mat, ax=ax)

ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
#ax.set_xticks(np.linspace(0, 80, 5))
ax.set_ylabel('Functional Interactions')
ax.set_xlabel('Subgraphs')

plt.savefig('./e02b-Figures/Subgraph-Cfg_Matrix.svg')
plt.close()      

# Plot subgraph adjacency
plt.figure()
n_row = np.floor(np.sqrt(n_fac))
n_col = np.ceil(n_fac / n_row)
for ii, subg in enumerate(fac_subnet):
    adj = convert_conn_vec_to_adj_matr(subg)

    ax = plt.subplot(n_row, n_col, ii+1)
    mat = ax.matshow(adj, cmap='rainbow', vmin=0, vmax=1)
    #plt.colorbar(mat, ax=ax)
    ax.set_axis_off()
    
plt.savefig('./e02b-Figures/Subgraph-Adj_Matrices.svg')
plt.show()
plt.close()      

# Plot Coefficients
plt.figure()
ax = plt.subplot(111)

fac_coef = fac_coef.T
norm_fac = fac_coef - fac_coef.mean(axis=0)
for ff in xrange(n_fac):
    ax.plot(ff + norm_fac[:, ff] / (3*np.std(norm_fac[:, ff])), color=[66/256., 152/256., 221./256])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylim([-1, n_fac+1])
ax.set_ylabel('Subgraphs')
ax.set_xlabel('Time Windows')

plt.savefig('./e02b-Figures/Subgraph-Coefs.svg')
plt.show()
plt.close()  

# Subgraphs of Brain Systems

## Load Subgraphs and Expression

In [ ]:
# Grab the subgraphs and expression from consensus NMF
df_nmf = np.load("{}/NMF_Optimization.consensus_subgraph.npz".format(path_ExpData),
                 mmap_mode='r')
fac_subnet = df_nmf['fac_subnet']
fac_coef = df_nmf['fac_coef']
n_fac = fac_subnet.shape[0]
n_conn = fac_subnet.shape[1]
n_node = np.int(np.ceil(np.sqrt(n_conn*2)))
n_obs = fac_coef.shape[1]

# Retrieve the configuration matrix
# Get expression for: subgraphs, subjects, task conditions + pos/neg interactions, blocks
path_cfg_expr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))[0]
df_cfg = np.load(path_cfg_expr, mmap_mode='r')
cfg_key = df_cfg['cfg_key']
key_type = np.unique(cfg_key)
n_key = len(key_type)
n_block = 6 
n_subj = n_obs / (n_key*n_block)
fac_coef_subj = np.zeros((n_fac, n_subj, n_key, n_block))
for key_ii, key in enumerate(key_type):
    key_ix = np.flatnonzero(cfg_key == key)
    fac_coef_subj[:, :, key_ii, :] = fac_coef[:, key_ix].reshape(n_fac, n_subj, n_block)

## Load Atlas

In [ ]:
### Get the Lausanne Labels
df_parcel = pd.read_csv('{}/LausanneScale125.csv'.format(path_CoreData))

lausanne_lbl = []
for lbl_id, lbl_roi, lbl_hemi in zip(df_parcel.Label_ID, df_parcel.ROI, df_parcel.Hemisphere):
    roi_name = '{}_{}'.format(lbl_hemi, lbl_roi)
    lausanne_lbl.append(roi_name)
lausanne_lbl = np.array(lausanne_lbl)

### Get the system assignments for each ROI
df_sys = h5py.File('{}/sysInfo234.mat'.format(path_CoreData), 'r')

system_lbl = [''.join(unichr(c) for c in df_sys[rr])
              for rr in df_sys['sysInfo']['system'][0, :]]
for ii in xrange(len(system_lbl), n_node):
    system_lbl.append(u'cerebellum')
system_lbl = np.array(system_lbl)

system_name = np.unique(system_lbl)
n_system = len(system_name)
sys_triu_ix, sys_triu_iy = np.triu_indices(n_system, k=0)
n_sys_conn = len(sys_triu_ix)

df_sys.close()

## Convert ROI Subgraphs to Brain System Subgraphs

In [ ]:
n_perm = 1000
alpha = 0.01

system_subgraph = []
for fac_i, subg in enumerate(fac_subnet):
    print('Processed: {} of {}'.format(fac_i+1, len(fac_subnet)))
    
    adj = convert_conn_vec_to_adj_matr(subg)
    
    temp_system_subg = np.zeros(n_sys_conn)
    for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):        
        ij_sys_ix = np.flatnonzero(system_lbl == system_name[tr_ix])
        ik_sys_ix = np.flatnonzero(system_lbl == system_name[tr_iy])  
        
        ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
        mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()
        
        temp_system_subg[tr_ii] = mean_conn

        
    null_system_subg = np.zeros((n_perm, n_sys_conn))
    for perm_i in xrange(n_perm):
        for tr_ii, (tr_ix, tr_iy) in enumerate(zip(sys_triu_ix, sys_triu_iy)):
            system_lbl_perm = np.random.permutation(system_lbl)
            ij_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_ix])
            ik_sys_ix = np.flatnonzero(system_lbl_perm == system_name[tr_iy])  

            ij_ik_sys_ix, ik_ij_sys_ix = np.meshgrid(ij_sys_ix, ik_sys_ix)
            mean_conn = adj[ij_ik_sys_ix, ik_ij_sys_ix].mean()

            null_system_subg[perm_i, tr_ii] = mean_conn
        
    pval_system_subg = np.mean(null_system_subg > temp_system_subg, axis=0)
    filt_system_subg = pval_system_subg.copy()
    filt_system_subg[pval_system_subg < alpha] = False
    filt_system_subg[pval_system_subg >= alpha] = True
    
    filt_ix = np.nonzero(filt_system_subg)
    thrsh_system_subg = temp_system_subg.copy()
    thrsh_system_subg[filt_ix] = 0
    
    temp_system_adj = np.zeros((n_system, n_system))
    temp_system_adj[sys_triu_ix, sys_triu_iy] = temp_system_subg
    temp_system_adj += np.triu(temp_system_adj, k=1).T

    thrsh_system_adj = np.zeros((n_system, n_system))
    thrsh_system_adj[sys_triu_ix, sys_triu_iy] = thrsh_system_subg
    thrsh_system_adj += np.triu(thrsh_system_adj, k=1).T
    
    
    # Format the system string output
    system_str = '\n'
    system_pair = []
    for sys_sig_ix in np.flatnonzero(filt_system_subg == 0):
        system_pair.append((system_name[sys_triu_ix[sys_sig_ix]],
                            system_name[sys_triu_iy[sys_sig_ix]]))
        system_str += '%20s <--> %-20s\n' % (system_pair[-1][0],
                                             system_pair[-1][1])
    
    # Generate subgraph dictionary
    system_subgraph.append({'Subgraph_ID': fac_i+1,
                            'filt_system_subg': thrsh_system_adj,
                            'system_pairs': system_pair,
                            'system_string': system_str,
                            'expr_coef': fac_coef_subj[fac_i, ...]})

np.savez('{}/Subgraph.11System.npz'.format(path_ExpData),
         system_subgraph=system_subgraph,
         system_labels=system_lbl,
         system_names=system_name,
         lausanne_labels=lausanne_lbl,
         task_key=key_type)

## Filter Subgraphs with Sparse Expression 

In [ ]:
# Load in the System-Level subgraph data
df_subg = np.load('{}/Subgraph.11System.npz'.format(path_ExpData))
sys_subgraph = df_subg['system_subgraph']
n_fac = len(sys_subgraph)

pct_sparse_mean = []
pct_sparse_std = []
for fac_ix in xrange(n_fac):
    fac_coef = sys_subgraph[fac_ix]['expr_coef']
    pct_sparse = (fac_coef == 0).reshape(fac_coef.shape[0], -1).mean(axis=-1)
    pct_sparse_mean.append(pct_sparse.mean())
    pct_sparse_std.append(pct_sparse.std() / np.sqrt(fac_coef.shape[0]))
pct_sparse_mean = np.array(pct_sparse_mean)
pct_sparse_std = np.array(pct_sparse_std)

# Find the sparsity order
fac_ord_ix = np.argsort(pct_sparse_mean)
fac_thresh = 20

# Plot the distribution of temporal sparsity over subgraphs
% matplotlib inline
plt.figure()
ax = plt.subplot(111)
ax.plot(np.arange(n_fac), pct_sparse_mean[fac_ord_ix], 'k')
ax.fill_between(np.arange(n_fac),
                pct_sparse_mean[fac_ord_ix]-pct_sparse_std[fac_ord_ix],
                pct_sparse_mean[fac_ord_ix]+pct_sparse_std[fac_ord_ix])
ax.vlines(fac_thresh, 0, 1, 'r')

ax.set_xlim([-0.5, n_fac-0.5])        
plt.xticks(np.arange(0, n_fac+1, 6),
           np.arange(0, n_fac+1, 6))
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Ranked Subgraphs')

ax.set_ylim([0, 1])        
ax.set_yticks(np.linspace(0, 1.0, 5))
ax.yaxis.set_ticks_position('left')
ax.set_ylabel('Percent Sparse Coefficients')
plt.savefig('./e02b-Figures/Sparse_Coefs.svg')
plt.show()

sys_subgraph = sys_subgraph[fac_ord_ix[:fac_thresh]]

np.savez('{}/Subgraph.11System.Filtered.npz'.format(path_ExpData),
         system_subgraph=sys_subgraph,
         system_labels=df_subg['system_labels'],
         system_names=df_subg['system_names'],
         lausanne_labels=df_subg['lausanne_labels'],
         task_key=df_subg['task_key'])

## Plot all Brain System Subgraphs

In [ ]:
df_subg = np.load('{}/Subgraph.11System.Filtered.npz'.format(path_ExpData))
sys_subgraph = df_subg['system_subgraph']

# Plot each result
for fac_ix in xrange(len(sys_subgraph)):
    f_sys_subgraph = sys_subgraph[fac_ix]
    
    print('\n\n\n')
    print('****************************** Subgraph {} ******************************'.format(fac_ix+1))    
    #print(f_sys_subgraph['system_string'])
    
    plt.figure(figsize=(4, 4))
    ax = plt.subplot(1, 1, 1)
    ax.matshow(f_sys_subgraph['filt_system_subg'], cmap='rainbow')
    if len(df_subg['system_names']) < 20:
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.set_yticks(np.arange(len(df_subg['system_names'])))
        ax.set_xticks(np.arange(len(df_subg['system_names'])))
        ax.set_yticklabels(df_subg['system_names'])
        ax.set_xticklabels(df_subg['system_names'])
        _ = [tick.label.set_fontsize(8) for tick in ax.yaxis.get_major_ticks()]
        _ = [tick.label.set_fontsize(8) for tick in ax.xaxis.get_major_ticks()]        
        _ = [tick.label.set_rotation(-75) for tick in ax.xaxis.get_major_ticks()]        
    else:
        ax.set_axis_off()
    plt.savefig('./e02b-Figures/System_Subgraph-{}.svg'.format(fac_ix+1))
    plt.close()
    #plt.show()